# Métodos de optimización no lineal sin restricciones
----------------

## Autores
* Quesada, Francisco
* Serpe, Octavio
* Arca, Gonzalo

-----------------

## Librerías utilizadas
* SciPy
* NumPy
-----------------


In [814]:
import math
from statistics import mean
from scipy.optimize import minimize
import numpy as np
import time
from autograd.wrap_util import wraps
from autograd.misc import flatten
import numdifftools as nd
import matplotlib.pyplot as plt

In [815]:
MIN_REAL = -10
MAX_REAL = 10

NUMBER_OF_PARAMETERS = 11

DATASET_INPUT = [(4.4793, -4.0765, -4.0765), (-4.1793, -
                                              4.9218, 1.7664), (-3.9429, -0.7689, 4.8830)]

DATASET_OUTPUT = [0, 1, 1]

error_GD = -1;
error_CG = -1;
error_Adam = -1;

x0 = np.random.uniform(low=MIN_REAL, high=MAX_REAL, size=NUMBER_OF_PARAMETERS)


In [816]:
# W 3 element vector
# w 2x3 matrix as 6 element vector
# w_0 2 element vector
# reactive_values 3 element vector
def f(W, w, w_0, reactive_values):
    inner_g_input = 0
    outer_g_input = 0

    for j in range(2):
        for k in range(3):
            inner_g_input += w[k + (j * 3)] * reactive_values[k] - w_0[j]
        outer_g_input += W[j] * g(inner_g_input)
    return g(outer_g_input - W[0])


# dataset_input 3 element vector
# dataset_output 3 element vector
def error(x, dataset_input, dataset_output):
    #dataset_input, dataset_output = dataset_tuple
    error = 0

    W = x[0:3]
    w = x[3:9]
    w_0 = x[9:11]

    for i in range(len(dataset_input)):
        error += math.pow(dataset_output[i] -
                          f(W, w, w_0, dataset_input[i]), 2)

    return error


def g(x):
    # If over the limit, return the limit value
    try:
        exp_value = math.exp(x)
    except OverflowError:
        return 1

    return exp_value / (1 + exp_value)

In [817]:
def print_output(x, time, error,iterations):
  print(f'Argumentos óptimos:\nW = \t{x[0:3]}\nw = \t{x[3:6]}\n\t{x[6:9]}\nw_0 = \t{x[9:11]}\n')
  print(f'error: {error}')
  print("Tiempo de ejecucion:", time, "segundos")
  print("Iteraciones:", iterations)


## Método 1: Gradiente descendiente (método quasi-Newton BFGS)

In [818]:
# https://docs.scipy.org/doc/scipy/reference/optimize.minimize-bfgs.html


def gradient_descent(x0,print=True):
  start_time = time.perf_counter()
  optimize_result_GD = minimize(error, x0, method='BFGS', args=(DATASET_INPUT, DATASET_OUTPUT), tol=1e-5, options={"disp":print})
  end_time = time.perf_counter()
  error_GD = error(optimize_result_GD.x, DATASET_INPUT, DATASET_OUTPUT)
  return optimize_result_GD.x,end_time-start_time,error_GD,optimize_result_GD.nit

optimal_x_GD,time_GD,error_GD,nit = gradient_descent(x0)
print_output(optimal_x_GD,time_GD,error_GD,nit)

Optimization terminated successfully.
         Current function value: 0.000003
         Iterations: 11
         Function evaluations: 168
         Gradient evaluations: 14
Argumentos óptimos:
W = 	[-6.76740124 -7.29465516 -0.04178769]
w = 	[ 3.09403315 -4.63234112 -8.92559705]
	[-5.26227584  7.49889802 -7.57303708]
w_0 = 	[ 4.2934149  -5.49070771]

error: 3.1026815251482652e-06
Tiempo de ejecucion: 0.00781695700425189 segundos
Iteraciones: 11


## Método 2: Gradientes conjugados

In [819]:
# https://docs.scipy.org/doc/scipy/reference/optimize.minimize-cg.html

def conjugate_gradient(x0,print=True):
  start_time = time.perf_counter()
  optimize_result_CG = minimize(error, x0, method='CG', args=(DATASET_INPUT, DATASET_OUTPUT), tol=1e-5, options={"disp":print})
  end_time = time.perf_counter()
  error_CG = error(optimize_result_CG.x, DATASET_INPUT, DATASET_OUTPUT)
  return optimize_result_CG.x,end_time-start_time,error_CG,optimize_result_CG.nit

optimal_x_CG,time_CG,error_CG,nit = conjugate_gradient(x0)

print_output(optimal_x_CG,time_CG,error_CG,nit)

Optimization terminated successfully.
         Current function value: 0.000004
         Iterations: 5
         Function evaluations: 372
         Gradient evaluations: 31
Argumentos óptimos:
W = 	[-6.5614418  -7.32068426 -0.04178769]
w = 	[ 3.09403315 -4.63234112 -8.92559679]
	[-5.26227584  7.49889802 -7.57303708]
w_0 = 	[ 4.29341489 -5.49070771]

error: 4.423919176399227e-06
Tiempo de ejecucion: 0.01187059700168902 segundos
Iteraciones: 5


## Método 3: ADAM

## Implementacion Método ADAM

In [823]:
def adam(f,grad, x,callback=None, num_iters=1000,
         step_size=0.001, b1=0.9, b2=0.999, eps=10**-8,tol=1e-5):
    """Adam as described in http://arxiv.org/pdf/1412.6980.pdf.
    It's basically RMSprop with momentum and some correction terms."""
    m = np.zeros(len(x))
    v = np.zeros(len(x))
    for i in range(num_iters):
        error = f(x,DATASET_INPUT,DATASET_OUTPUT)
        if error < tol:
            break;
        g = grad(x, i)
        if callback: callback(x, i, g)
        m = (1 - b1) * g      + b1 * m  # First  moment estimate.
        v = (1 - b2) * (g**2) + b2 * v  # Second moment estimate.
        mhat = m / (1 - b1**(i + 1))    # Bias correction.
        vhat = v / (1 - b2**(i + 1))
        x = x - step_size*mhat/(np.sqrt(vhat) + eps)
    return x,i

In [824]:
x0 = np.random.uniform(low=MIN_REAL, high=MAX_REAL, size=NUMBER_OF_PARAMETERS)
gradient = nd.Gradient(error)

def grad(x,i):
    return gradient(x, DATASET_INPUT, DATASET_OUTPUT)


def adam_optimization(x0):
    start_time = time.perf_counter()
    optimize_result_Adam,steps= adam(error,grad,x0,step_size= 0.8)
    error_Adam = error(optimize_result_Adam, DATASET_INPUT, DATASET_OUTPUT)
    end_time = time.perf_counter()
    return optimize_result_Adam,end_time-start_time,error_Adam,steps


optimize_result_Adam,time_Adam,error_Adam,steps = adam_optimization(x0)

print_output(optimize_result_Adam, time_Adam, error_Adam,steps)


Argumentos óptimos:
W = 	[-0.69314718 -4.36545728 -9.99719673]
w = 	[7.88655693 9.97619305 7.12451318]
	[ 7.35512871  1.93672014 -1.77064303]
w_0 = 	[8.5321997  1.21864877]

error: 0.6666666666666505
Tiempo de ejecucion: 6.083232453005621 segundos
Iteraciones: 999
